# Recipe Dataset Preprocessing for RAG

This notebook processes recipe data to create a clean, semantically meaningful dataset optimized for Retrieval-Augmented Generation (RAG) systems.

## Overview

The preprocessing pipeline:
1. Loads recipe data from a Parquet file
2. Creates a `search_vector` column with clean, semantic text for embedding
3. Removes unnecessary columns
4. Saves the processed dataset ready for RAG

## Requirements

- `pyarrow` for reading Parquet files
- `pandas` for data manipulation
- `numpy` for array handling

In [48]:
# Install required packages
%pip install pyarrow pandas numpy


Note: you may need to restart the kernel to use updated packages.


## 1. Setup and Imports

Configure environment and import necessary libraries.


In [49]:
import os
import re
import json
import ast
import numpy as np
import pandas as pd
import pyarrow as pa

# IMPORTANT: Set environment variable BEFORE importing pyarrow/pandas
# This tells pyarrow to ignore Python extension types when reading
os.environ['PYARROW_IGNORE_PYTHON_EXTENSION_TYPES'] = '1'

# Unregister pandas extension types if they're already registered
# This prevents the "already defined" error when pandas tries to register them
pandas_ext_types = ['pandas.period', 'pandas.interval', 'pandas.nullable_int', 
                    'pandas.nullable_bool', 'pandas.nullable_float', 
                    'pandas.nullable_string', 'pandas.nullable_bytes']
for ext_name in pandas_ext_types:
    try:
        pa.unregister_extension_type(ext_name)
    except (KeyError, ValueError):
        pass  # Extension type not registered, which is fine

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2. Load Data

Load the recipe dataset from the Parquet file.
File is downloaded from url: https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews?resource=download&select=recipes.parquet

In [50]:
# Load the recipe dataset
df = pd.read_parquet("recipes.parquet", engine="pyarrow")

print(f"✓ Dataset loaded successfully")
print(f"  Shape: {df.shape}")
print(f"  Columns: {len(df.columns)}")
print(f"\nColumn names:")
print(df.columns.tolist())


✓ Dataset loaded successfully
  Shape: (522517, 28)
  Columns: 28

Column names:
['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime', 'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent', 'RecipeServings', 'RecipeYield', 'RecipeInstructions']


## 3. Data Exploration

Quick exploration of the dataset structure and data types.


In [51]:
# Display dataset info
print("Dataset Information:")
print("=" * 80)
df.info()
print("\n" + "=" * 80)

# Check data types for list-like columns
print("\nSample data types for list columns:")
sample_row = df.iloc[0]
print(f"RecipeIngredientParts: {type(sample_row['RecipeIngredientParts'])}")
print(f"Keywords: {type(sample_row['Keywords'])}")
print(f"RecipeInstructions: {type(sample_row['RecipeInstructions'])}")

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   RecipeId                    522517 non-null  float64            
 1   Name                        522517 non-null  object             
 2   AuthorId                    522517 non-null  int32              
 3   AuthorName                  522517 non-null  object             
 4   CookTime                    439972 non-null  object             
 5   PrepTime                    522517 non-null  object             
 6   TotalTime                   522517 non-null  object             
 7   DatePublished               522517 non-null  datetime64[us, UTC]
 8   Description                 522512 non-null  object             
 9   Images                      522516 non-null  object             
 10  RecipeCategory         

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   RecipeId                    522517 non-null  float64            
 1   Name                        522517 non-null  object             
 2   AuthorId                    522517 non-null  int32              
 3   AuthorName                  522517 non-null  object             
 4   CookTime                    439972 non-null  object             
 5   PrepTime                    522517 non-null  object             
 6   TotalTime                   522517 non-null  object             
 7   DatePublished               522517 non-null  datetime64[us, UTC]
 8   Description                 522512 non-null  object             
 9   Images                      522516 non-null  object             
 10  RecipeCategory              521766 non-null 

In [53]:
# Inspect data types and sample values for list-like columns
import ast
import json

# Check if columns are stored as strings (JSON) or actual lists
sample_row = df.iloc[0]

print("Sample RecipeIngredientParts:", type(sample_row['RecipeIngredientParts']))
print("Sample Keywords:", type(sample_row['Keywords']))
print("Sample RecipeInstructions:", type(sample_row['RecipeInstructions']))
print("\nFirst few characters of each:")
print("RecipeIngredientParts:", str(sample_row['RecipeIngredientParts'])[:200])
print("Keywords:", str(sample_row['Keywords'])[:200])
print("RecipeInstructions:", str(sample_row['RecipeInstructions'])[:200])


Sample RecipeIngredientParts: <class 'numpy.ndarray'>
Sample Keywords: <class 'numpy.ndarray'>
Sample RecipeInstructions: <class 'numpy.ndarray'>

First few characters of each:
RecipeIngredientParts: ['blueberries' 'granulated sugar' 'vanilla yogurt' 'lemon juice']
Keywords: ['Dessert' 'Low Protein' 'Low Cholesterol' 'Healthy' 'Free Of...' 'Summer'
 'Weeknight' 'Freezer' 'Easy']
RecipeInstructions: ['Toss 2 cups berries with sugar.'
 'Let stand for 45 minutes, stirring occasionally.'
 'Transfer berry-sugar mixture to food processor.'
 'Add yogurt and process until smooth.'
 "Strain through fine 


## 4. Preprocessing Functions

Define helper functions for data preprocessing.


In [54]:
# Preprocess data for RAG
import re
import numpy as np

def safe_eval_list(value):
    """Safely convert string representation of list to actual list"""
    # Handle numpy arrays (common in parquet files)
    if isinstance(value, np.ndarray):
        return value.tolist() if value.size > 0 else []
    
    # Handle None or NaN
    if safe_isna(value):
        return []
    
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            # Try JSON first
            parsed = json.loads(value)
            if isinstance(parsed, list):
                return parsed
        except:
            pass
        try:
            # Try ast.literal_eval
            parsed = ast.literal_eval(value)
            if isinstance(parsed, list):
                return parsed
        except:
            pass
        # If it's a string but not a list, return as single-item list
        return [value] if value.strip() else []
    return []

def safe_isna(value):
    """Safely check if value is NaN, handling arrays and scalars"""
    if value is None:
        return True
    if isinstance(value, np.ndarray):
        return value.size == 0
    try:
        return pd.isna(value)
    except (ValueError, TypeError):
        return False

def format_time(time_str):
    """Format time string for search - converts ISO 8601 duration to human-readable format"""
    if safe_isna(time_str):
        return ""
    # Handle empty strings
    if not time_str or (isinstance(time_str, str) and not time_str.strip()):
        return ""
    
    time_str = str(time_str).strip()
    
    # Parse ISO 8601 duration format (PT45M, PT24H, PT24H45M, etc.)
    if time_str.startswith('PT'):
        # Remove PT prefix
        duration = time_str[2:]
        parts = []
        
        # Extract hours
        hour_match = re.search(r'(\d+)H', duration)
        hours = int(hour_match.group(1)) if hour_match else 0
        
        # Extract minutes
        minute_match = re.search(r'(\d+)M', duration)
        minutes = int(minute_match.group(1)) if minute_match else 0
        
        # Extract seconds (if present)
        second_match = re.search(r'(\d+)S', duration)
        seconds = int(second_match.group(1)) if second_match else 0
        
        # Format as human-readable
        time_parts = []
        if hours > 0:
            time_parts.append(f"{hours} hour{'s' if hours != 1 else ''}")
        if minutes > 0:
            time_parts.append(f"{minutes} minute{'s' if minutes != 1 else ''}")
        if seconds > 0 and hours == 0 and minutes == 0:  # Only show seconds if no hours/minutes
            time_parts.append(f"{seconds} second{'s' if seconds != 1 else ''}")
        
        if time_parts:
            return " ".join(time_parts)
        else:
            return ""  # Empty duration
    
    # If not ISO 8601 format, return as-is
    return time_str

def infer_dietary_tags(ingredients_list, keywords_list):
    """Infer dietary restrictions/preferences from ingredients and keywords"""
    tags = []
    ingredients_text = " ".join([str(ing).lower() for ing in ingredients_list])
    keywords_text = " ".join([str(kw).lower() for kw in keywords_list])
    combined_text = ingredients_text + " " + keywords_text
    
    # Vegetarian/Vegan
    meat_keywords = ['chicken', 'beef', 'pork', 'lamb', 'turkey', 'fish', 'seafood', 'meat', 'bacon', 'sausage']
    dairy_keywords = ['milk', 'cheese', 'butter', 'cream', 'yogurt', 'dairy']
    egg_keywords = ['egg', 'eggs']
    
    has_meat = any(keyword in combined_text for keyword in meat_keywords)
    has_dairy = any(keyword in combined_text for keyword in dairy_keywords)
    has_eggs = any(keyword in combined_text for keyword in egg_keywords)
    
    if not has_meat and not has_dairy and not has_eggs:
        tags.append("vegan")
    elif not has_meat:
        tags.append("vegetarian")
    
    # Gluten-free
    gluten_keywords = ['wheat', 'flour', 'bread', 'pasta', 'gluten']
    has_gluten = any(keyword in combined_text for keyword in gluten_keywords)
    if not has_gluten or 'gluten-free' in combined_text:
        tags.append("gluten-free")
    
    # Low-carb / Keto
    if 'low-carb' in combined_text or 'keto' in combined_text:
        tags.append("low-carb")
    
    # Other common dietary tags
    if 'paleo' in combined_text:
        tags.append("paleo")
    if 'keto' in combined_text:
        tags.append("keto")
    if 'dairy-free' in combined_text:
        tags.append("dairy-free")
    
    return tags

def format_nutrition(row):
    """Format nutritional information as searchable text"""
    nutrition_parts = []
    
    if not safe_isna(row['Calories']) and row['Calories'] > 0:
        nutrition_parts.append(f"{int(row['Calories'])} calories")
    
    macros = []
    if not safe_isna(row['ProteinContent']) and row['ProteinContent'] > 0:
        macros.append(f"{row['ProteinContent']:.1f}g protein")
    if not safe_isna(row['CarbohydrateContent']) and row['CarbohydrateContent'] > 0:
        macros.append(f"{row['CarbohydrateContent']:.1f}g carbs")
    if not safe_isna(row['FatContent']) and row['FatContent'] > 0:
        macros.append(f"{row['FatContent']:.1f}g fat")
    
    if macros:
        nutrition_parts.append(", ".join(macros))
    
    # Dietary restrictions based on content
    if not safe_isna(row['SodiumContent']) and row['SodiumContent'] < 500:
        nutrition_parts.append("low-sodium")
    if not safe_isna(row['SugarContent']) and row['SugarContent'] < 10:
        nutrition_parts.append("low-sugar")
    if not safe_isna(row['FiberContent']) and row['FiberContent'] > 5:
        nutrition_parts.append("high-fiber")
    
    return " | ".join(nutrition_parts) if nutrition_parts else ""



## 5. Create Search Vector

Generate a clean, semantically meaningful `search_vector` column optimized for RAG embeddings.

The search vector contains:
- Recipe name (lowercase)
- Semantic tags (category, keywords, dietary info, health descriptors)
- Ingredients (cleaned, lowercase, no quantities)

**Note:** All numeric metadata (times, calories, grams, servings) and structural labels are removed to focus on semantic meaning.


In [55]:
def create_search_vector(row):
    """Create clean, semantically meaningful search vector for RAG embedding"""
    parts = []
    
    # 1. Recipe name (first sentence)
    recipe_name = ""
    if not safe_isna(row['Name']):
        recipe_name = str(row['Name']).strip().lower()
        parts.append(recipe_name)
    
    # 2. Collect all semantic tags and descriptors
    semantic_tags = []
    
    # Category
    if not safe_isna(row['RecipeCategory']):
        category = str(row['RecipeCategory']).strip().lower()
        semantic_tags.append(category)
    
    # Keywords (lowercase, clean)
    keywords = safe_eval_list(row['Keywords'])
    if keywords:
        for kw in keywords:
            kw_clean = str(kw).strip().lower()
            if kw_clean and kw_clean not in semantic_tags:
                semantic_tags.append(kw_clean)
    
    # Dietary tags (inferred)
    ingredients = safe_eval_list(row['RecipeIngredientParts'])
    dietary_tags = infer_dietary_tags(ingredients, keywords)
    for tag in dietary_tags:
        if tag not in semantic_tags:
            semantic_tags.append(tag)
    
    # Health descriptors from nutrition (no numbers, just descriptors)
    if not safe_isna(row['SodiumContent']) and row['SodiumContent'] < 500:
        if "low-sodium" not in semantic_tags:
            semantic_tags.append("low-sodium")
    if not safe_isna(row['SugarContent']) and row['SugarContent'] < 10:
        if "low-sugar" not in semantic_tags:
            semantic_tags.append("low-sugar")
    if not safe_isna(row['FiberContent']) and row['FiberContent'] > 5:
        if "high-fiber" not in semantic_tags:
            semantic_tags.append("high-fiber")
    if not safe_isna(row['FatContent']) and row['FatContent'] < 10:
        if "low-fat" not in semantic_tags:
            semantic_tags.append("low-fat")
    
    # Add semantic tags as second part (semicolon-separated)
    if semantic_tags:
        parts.append("; ".join(semantic_tags))
    
    # 3. Ingredients (lowercase, no quantities)
    if ingredients:
        ingredients_lower = [str(ing).strip().lower() for ing in ingredients]
        # Remove common quantity words and numbers
        cleaned_ingredients = []
        for ing in ingredients_lower:
            # Remove leading numbers, measurements, etc.
            ing_clean = re.sub(r'^\d+\s*', '', ing)  # Remove leading numbers
            ing_clean = re.sub(r'^\d+/\d+\s*', '', ing_clean)  # Remove fractions
            ing_clean = re.sub(r'\b(cup|cups|tbsp|tsp|oz|lb|g|kg|ml|l|tablespoon|teaspoon|ounce|pound|gram|kilogram|milliliter|liter)\b', '', ing_clean)
            ing_clean = re.sub(r'\s+', ' ', ing_clean).strip()  # Clean whitespace
            if ing_clean:
                cleaned_ingredients.append(ing_clean)
        
        if cleaned_ingredients:
            # Format as natural sentence
            if len(cleaned_ingredients) == 1:
                parts.append(f"ingredients include {cleaned_ingredients[0]}.")
            elif len(cleaned_ingredients) == 2:
                parts.append(f"ingredients include {cleaned_ingredients[0]} and {cleaned_ingredients[1]}.")
            else:
                ingredients_str = ", ".join(cleaned_ingredients[:-1]) + f", and {cleaned_ingredients[-1]}"
                parts.append(f"ingredients include {ingredients_str}.")
    
    # Join all parts with periods or semicolons (already handled above)
    search_text = ". ".join(parts)
    
    # Clean up: remove multiple spaces, ensure proper punctuation
    search_text = re.sub(r'\.+', '.', search_text)  # Multiple periods to one
    search_text = re.sub(r'\s+', ' ', search_text)  # Multiple spaces to one
    search_text = search_text.strip()
    
    # Ensure it ends with a period
    if search_text and not search_text.endswith('.'):
        search_text += "."
    
    return search_text

# Create a copy for processing
df_processed = df.copy()

print("Creating search vector column...")
df_processed['search_vector'] = df_processed.apply(create_search_vector, axis=1)

print(f"Search vector created. Sample:")
print(df_processed['search_vector'].iloc[0][:500])


Creating search vector column...
Search vector created. Sample:
low-fat berry blue frozen dessert. frozen desserts; dessert; low protein; low cholesterol; healthy; free of.; summer; weeknight; freezer; easy; vegetarian; gluten-free; low-sodium; low-fat. ingredients include blueberries, granulated sugar, vanilla yogurt, and lemon juice.


## 6. Remove Unnecessary Columns

Keep only columns needed for RAG and remove metadata that's not useful for semantic search.


In [56]:
# Remove unnecessary columns and keep only what's needed for RAG
columns_to_keep = [
    'RecipeId',           # For identification
    'Name',               # Recipe name
    'Description',        # Recipe description
    'RecipeCategory',     # Category
    'RecipeIngredientParts',  # Ingredients list
    'RecipeInstructions', # Cooking instructions
    'PrepTime',           # Preparation time
    'CookTime',           # Cooking time
    'TotalTime',          # Total time
    'RecipeServings',     # Number of servings
    'Calories',           # Nutritional info
    'FatContent',
    'SaturatedFatContent',
    'CholesterolContent',
    'SodiumContent',
    'CarbohydrateContent',
    'FiberContent',
    'SugarContent',
    'ProteinContent',
    'Keywords',           # Keywords/tags
    'search_vector'       # The main RAG search column
]

# Create final dataset
df_rag = df_processed[columns_to_keep].copy()

print(f"Original columns: {len(df.columns)}")
print(f"Final columns: {len(df_rag.columns)}")
print(f"\nRemoved columns: {set(df.columns) - set(df_rag.columns)}")
print(f"\nFinal dataset shape: {df_rag.shape}")
print(f"\nFinal columns:")
print(df_rag.columns.tolist())


Original columns: 28
Final columns: 21

Removed columns: {'RecipeIngredientQuantities', 'AuthorName', 'AggregatedRating', 'DatePublished', 'RecipeYield', 'AuthorId', 'Images', 'ReviewCount'}

Final dataset shape: (522517, 21)

Final columns:
['RecipeId', 'Name', 'Description', 'RecipeCategory', 'RecipeIngredientParts', 'RecipeInstructions', 'PrepTime', 'CookTime', 'TotalTime', 'RecipeServings', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent', 'Keywords', 'search_vector']


## 7. Preview Results

Preview a sample of the processed data to verify the search_vector format.


In [57]:
# Preview the processed data
print("Sample processed row:")
print("=" * 80)
sample = df_rag.iloc[0]
print(f"Recipe ID: {sample['RecipeId']}")
print(f"Name: {sample['Name']}")
print(f"\nSearch Vector (first 500 chars):")
print(sample['search_vector'][:500])
print("\n...")
print(f"\nFull search vector length: {len(sample['search_vector'])} characters")
print("\n" + "=" * 80)


Sample processed row:
Recipe ID: 38.0
Name: Low-Fat Berry Blue Frozen Dessert

Search Vector (first 500 chars):
low-fat berry blue frozen dessert. frozen desserts; dessert; low protein; low cholesterol; healthy; free of.; summer; weeknight; freezer; easy; vegetarian; gluten-free; low-sodium; low-fat. ingredients include blueberries, granulated sugar, vanilla yogurt, and lemon juice.

...

Full search vector length: 273 characters



## 8. Data Quality Check

Verify data quality and completeness of the processed dataset.


In [58]:
# Check data quality
print("Data Quality Check:")
print(f"Total rows: {len(df_rag)}")
print(f"Rows with search_vector: {df_rag['search_vector'].notna().sum()}")
print(f"Rows with empty search_vector: {(df_rag['search_vector'] == '').sum()}")
print(f"Average search_vector length: {df_rag['search_vector'].str.len().mean():.0f} characters")
print(f"\nMissing values per column:")
print(df_rag.isnull().sum().sort_values(ascending=False))


Data Quality Check:
Total rows: 522517
Rows with search_vector: 522517
Rows with empty search_vector: 0
Average search_vector length: 248 characters

Missing values per column:
RecipeServings           182911
CookTime                  82545
RecipeCategory              751
Description                   5
RecipeId                      0
CholesterolContent            0
Keywords                      0
ProteinContent                0
SugarContent                  0
FiberContent                  0
CarbohydrateContent           0
SodiumContent                 0
Calories                      0
SaturatedFatContent           0
FatContent                    0
Name                          0
TotalTime                     0
PrepTime                      0
RecipeInstructions            0
RecipeIngredientParts         0
search_vector                 0
dtype: int64


## 9. Save Processed Dataset

Save the processed dataset to a new Parquet file for use in RAG systems.


In [59]:
# Save the processed dataset for RAG
output_file = "recipes_rag_ready.parquet"
df_rag.to_parquet(output_file, engine='pyarrow', index=False)
print(f"Processed dataset saved to: {output_file}")
print(f"File size: {os.path.getsize(output_file) / (1024*1024):.2f} MB")
print(f"\nDataset is ready for RAG! Use the 'search_vector' column for semantic search.")


Processed dataset saved to: recipes_rag_ready.parquet
File size: 290.06 MB

Dataset is ready for RAG! Use the 'search_vector' column for semantic search.


## Summary

### What Was Done

1. **Created `search_vector` column**: A clean, semantically meaningful text field containing:
   - Recipe name (lowercase)
   - Semantic tags (category, keywords, dietary restrictions, health descriptors)
   - Ingredients (cleaned, lowercase, quantities removed)

2. **Removed unnecessary columns**:
   - AuthorId, AuthorName (not needed for search)
   - DatePublished (not relevant for recipe search)
   - Images (not needed for text-based RAG)
   - RecipeIngredientQuantities (redundant)
   - RecipeYield (redundant with RecipeServings)
   - AggregatedRating, ReviewCount (not needed for search)

3. **Kept essential columns** for:
   - Recipe identification (RecipeId)
   - Display (Name, Description, RecipeCategory)
   - Ingredients and instructions (for full recipe display)
   - Nutritional data (for filtering and display)
   - Time information (for filtering)
   - Keywords (for additional metadata)

### The `search_vector` Column

The search vector is optimized for semantic embedding and enables:
- **Ingredient-based queries**: "recipes with chicken and tomatoes"
- **Dietary restrictions**: "vegan recipes", "gluten-free desserts"
- **Health descriptors**: "low-sodium", "high-fiber", "low-fat"
- **Category/Cuisine**: "Italian pasta recipes", "breakfast recipes"
- **Combined queries**: "vegan high-protein quick meals"

All numeric metadata (times, calories, grams, servings) and structural labels are removed to focus purely on semantic meaning for better embedding quality.
